# practice-20-09

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, Annotated, Literal
from fastapi import FastAPI, Query, Path, HTTPException
import json


app = FastAPI()
class CreatePatient(BaseModel):

    patient_id: Annotated[
        str, Field(..., description="ID of the patient in DB", example="P001")
    ]
    name: Annotated[
        str, Field(..., description="Name of the patient", example="John Doe")
    ]
    city: Annotated[
        str, Field(..., description="City of the patient", example="New York")
    ]
    age: Annotated[int, Field(..., description="Age of the patient", example=30)]
    gender: Annotated[
        str, Field(..., description="Gender of the patient", example="Male")
    ]
    height: Annotated[
        float, Field(..., description="Height of the patient", example=1.75)
    ]
    weight: Annotated[
        float, Field(..., description="Weight of the patient", example=70.5)
    ]


class PatientUpdate(BaseModel):

    name: Annotated[Optional[str], Field(default=None)]
    city: Annotated[Optional[str], Field(default=None)]
    age: Annotated[Optional[int], Field(default=None, gt=0)]
    gender: Annotated[Optional[Literal['male', 'female']], Field(default=None)]
    height: Annotated[Optional[float], Field(default=None, gt=0)]
    weight: Annotated[Optional[float], Field(default=None, gt=0)]


def load_data():
    with open("patients.json", "r") as f:
        data = json.load(f)

    return data


def save_data(data):
    with open("patients.json", "w") as f:
        json.dump(data, f)

@app.put('/edit/{patient_id}')
def update_patient(patient_id: str, patient_update: PatientUpdate):
    data = load_data()

    if patient_id not in data:
        raise HTTPException(status_code=404, detail='Patient not found')

    existing_patient_info = data[patient_id]

    updated_patient_info = patient_update.model_dump(exclude_unset=True)

    for key, val in updated_patient_info.items():
        existing_patient_info[key] = val

    existing_patient_info['patient_id'] = patient_id
    patient_pydantic_object = CreatePatient(**existing_patient_info)
    existing_patient_info = patient_pydantic_object.model_dump(exclude="patient_id")

    data[patient_id] = existing_patient_info

    save_data(data)

In [ ]:
from fastapi.responses import JSONResponse

@app.delete('/delete/{patient_id}')
def delete_patient(patient_id: str):

    data = load_data()
    if patient_id not in data:
        raise HTTPException(status_code=404, detail='patient not found')

    del data[patient_id]

    save_data(data)

    return JSONResponse(
        status_code=200, content={"message": "Patient deleted successfully"}
    )